In [0]:
pde = spark.read.parquet("/mnt/bronze/pde/Prescription_Drug_Events.parquet")

In [0]:
from pyspark.sql.functions import col, to_date, year, when, lit

def standardize_columns(df):
    return df.toDF(*[c.lower() for c in df.columns])

# Clean PDE
pde_clean = (
    pde.dropDuplicates(["DESYNPUF_ID", "PDE_ID"])
    .withColumn("SRVC_DT", to_date(col("SRVC_DT"), "yyyyMMdd"))
    .fillna(0)
)

pde_clean = standardize_columns(pde_clean)

In [0]:
pde_clean.write.mode("overwrite").parquet("/mnt/silver/pde/pde_clean")

In [0]:
from pyspark.sql.functions import col, year, when, to_date, lit

# PDE Silver Table
pde_silver = pde_clean.select(
    "desynpuf_id", "pde_id", "srvc_dt", "prod_srvc_id",
    "qty_dspnsd_num", "tot_rx_cst_amt", "ptnt_pay_amt"
)

In [0]:
pde_silver.write.mode("overwrite").parquet("/mnt/silver/pde/pde_silver")